In [39]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


In [55]:
starbucks_response = requests.get("https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state")

In [56]:
from bs4 import BeautifulSoup
starbs_soup = BeautifulSoup(starbucks_response.content, "html.parser")

In [32]:
len(soup.find_all("table"))

1

In [63]:
table=(starbs_soup.find_all("table"))[0]
table

<table class="wpr-table"><thead class="table-head bg-wpr-table_header_bg relative z-40 px-0 py-0 text-xs uppercase tracking-wider"><tr><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 pin left-0 z-50 border-l-0"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">State</div></div></th><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 z-40"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">Starbucks Stores 2023<svg class="icon icon-tabler icon-tabler-chevron-down" fill="none" height="14" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" viewbox="0 0 24

In [48]:
#row
line=table.find_all("tr")

In [57]:
#state
row.find("th").text

'Vermont'

In [45]:
#num of starbs in state
row.find("td").text

'8'

In [78]:
starbs= []

# iterate over all rows in the population table
for row in table.find_all("tr")[1:]:

  #state
  state= row.find("th").text

  #num starbs
  num=row.find("td").text

  starbs.append({
                "state": state,
                "Starbucks Stores": num,
  })

  

In [79]:
starbs=pd.DataFrame(starbs)

In [80]:
dunkin_response = requests.get("https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state")

In [81]:
from bs4 import BeautifulSoup
dunks_soup = BeautifulSoup(dunkin_response.content, "html.parser")

In [82]:
dunkin_table=(dunks_soup.find_all("table"))[0]
dunkin_table

<table class="wpr-table"><thead class="table-head bg-wpr-table_header_bg relative z-40 px-0 py-0 text-xs uppercase tracking-wider"><tr><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 pin left-0 z-50 border-l-0"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">State</div></div></th><th class="datatable-th bg-wpr-table_header_bg text-wpr-table_header border-wpr-table_border sticky top-0 border-b bg-clip-padding px-3 py-2 text-left align-bottom leading-4 md:px-4 z-40"><div class="flex flex-col items-stretch justify-start"><div class="flex flex-row items-center cursor-pointer select-none">Dunkin Locations 2024<svg class="icon icon-tabler icon-tabler-chevron-down" fill="none" height="14" stroke="currentColor" stroke-linecap="round" stroke-linejoin="round" stroke-width="2" viewbox="0 0 24

In [83]:
dunky= []

# iterate over all rows in the population table
for row in dunkin_table.find_all("tr")[1:]:

  #state
  state= row.find("th").text

  #num starbs
  num=row.find("td").text

  dunky.append({
                "state": state,
                "Dunkin Stores": num,
  })

In [84]:
dunky =pd.DataFrame(dunky)

In [85]:
#combine
merge_df = pd.merge(starbs, dunky, how="inner", on='state')

In [ ]:
merge_df

In [119]:
#wiki table
wiki_response = requests.get("https://simple.wikipedia.org/wiki/List_of_U.S._states_by_population")

In [120]:
from bs4 import BeautifulSoup
wiki_soup = BeautifulSoup(wiki_response.content, "html.parser")

In [122]:
wiki_table=(wiki_soup.find_all("table"))[0]
#wiki_table


In [132]:
row =wiki_table.find_all("tr")[1]
#row

In [133]:

cell= row.find_all('td')
cell[2].get_text(strip=True)

'California'

In [ ]:
state= cells[0].text
    city = city.rstrip(city[-1:])
    if city.endswith("]"):
      city = city.rstrip(city[-3:])

In [142]:
wiki = []

# iterate over all rows in the population table
for row in wiki_table.find_all("tr")[1:]:


    # Get all the cells (<td>) in the row.
    #state
    cells = row.find_all("td")
    state=cells[2].get_text(strip=True)


 #find population
    pop = cells[3].get_text(strip=True)


    # Append this data.
    wiki.append({
        "State": state,
        "Population": pop,
})

In [143]:
pd.DataFrame(wiki)

,State,Population
0,California,"39,538,223"
1,Texas,"30,145,505"
2,Florida,"21,538,187"
3,New York,"20,201,249"
4,Pennsylvania,"13,002,700"
5,Illinois,"12,812,508"
6,Ohio,"11,799,448"
7,Georgia,"10,711,908"
8,North Carolina,"10,439,388"
9,Michigan,"10,077,331"
